In [1]:
import torch
import torchvision
import torch.nn as nn
import time
import json
import datetime
import matplotlib.pyplot as plt

In [2]:
%cd
from DeepLearning.Project2.data_loading_preparation import load_audio_dataloaders_validation

/home/kacper


In [3]:
def get_generic_classifier(input_size, hidden_size, output_size):
    classifier = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(hidden_size, output_size),
    )
    return classifier
    
class LSTMSimple(nn.Module):
    def __init__(
        self,
        input_size, 
        hidden_size,
        num_layers,
        num_classes = 12,
        avgpool_dim = 32,
        classifier_size = 512,
        add_dropout=True
    ):       
        super().__init__()
        self.normalization = nn.BatchNorm1d(input_size)
        self.features = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.avgpool = nn.AdaptiveAvgPool1d(avgpool_dim)
        self.classifier = get_generic_classifier(
            hidden_size * avgpool_dim,
            classifier_size,
            num_classes
        )

    def forward(self, x):
        x = self.normalization(x)
        x = x.mT
        x, _ = self.features(x)
        x = x.mT
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [4]:
def eval_accuracy(model, dataloader, reccurrent, training_device='cuda'):
    with torch.no_grad():
        model.to(training_device)
        correct = 0
        all_so_far = 0
        for inputs, labels in dataloader:
            inputs, labels = inputs.float().to(training_device), labels.float().to(training_device)
            yhat = model(inputs)
            pred = torch.argmax(yhat, dim=1)

            
            all_so_far += labels.size().numel()
            correct += torch.sum(pred.eq(labels)).item()
    return correct/all_so_far

In [5]:
def backup_to_ram(model):
    from copy import deepcopy
    return deepcopy(model).cpu()

class EarlyStopper:
    def __init__(self, patience = 3, backup_method=backup_to_ram):
        self.patience = patience
        self.current = 0
        
        self.backup_method = backup_method
        
        self.best_backup = None
        self.best_accuracy = 0.

    def should_continue(self, accuracy, model = None):
        if self.best_accuracy < accuracy:
            self.current = 0
            self.best_accuracy = accuracy
            if model is not None:
                self.best_backup = self.backup_method(model)
            return True
        
        self.current += 1
        
        if self.current >= self.patience:
            return False
        return True

In [6]:
def run_experiment(experiment_name, train_func, run, train_params=None):
    path = f"experiments_rnn/{experiment_name}_run_{run}_"
    print("Running experiment for ", path[:-1])
    
    import os
    try:
        if os.stat(path + "report.json").st_size != 0:
            print("Report exists already for " + path[:-1] + ". Skipping...")
            return
    except OSError:
        pass
    
    model, trajectory, validation_accuracy = train_func(train_params)
    
    with open(path + "report.json", "w") as f:
        json.dump(
            {
                "name": experiment_name,
                "train_params": train_params,
                "run": run,
                "best_accuracy_validation": validation_accuracy,
                "time_generated": datetime.datetime.now().isoformat(),
                "trajectory": trajectory
            },
            f
        )
    torch.save(model, path + "model.pt")

In [7]:
training_device = "cuda"
device = "cuda"
max_epochs = 250

def oneLSTM(train_params):  
    train, test, val = load_audio_dataloaders_validation(bs=1024, limit_11=0.7)
    criterion_weights = torch.tensor([1.] * 11 + [0.1]).to(device)
    model = LSTMSimple(**train_params).to(training_device)
    criterion = torch.nn.CrossEntropyLoss(weight=criterion_weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, betas=[0.9, 0.999], eps=10e-8)
    early_stopper = EarlyStopper(patience = 20)
    trajectory = []
    model.to(training_device)

    for epoch in range(1, max_epochs+1):
        model.train()
        for x, y in train:
            optimizer.zero_grad()
            x, y = x.float().to(device), y.float().to(device)
            (yhat) = model(x)
            loss = criterion(yhat.softmax(1), y.long())
            loss.backward()
            optimizer.step()
        
        validation_accuracy = eval_accuracy(model, val, training_device)
        
        print("Epoch: {}, Accuracy on validation set: {}".format(epoch, validation_accuracy))
        
        trajectory.append({
            "epoch": epoch,
            "validation": validation_accuracy,
        })
        
        if not early_stopper.should_continue(validation_accuracy, model):
            print("Early stop")
            model = early_stopper.best_backup
            model = model.to(device)
            break

    return model, trajectory, validation_accuracy


In [8]:
param_grid = {
    "small": {
        "hidden_size": 64,
        "input_size": 20,
        "num_layers": 1,
    },
    "smalldouble": {
        "hidden_size": 64,
        "input_size": 20,
        "num_layers": 1,
    },
    "tinydouble": {
        "hidden_size": 40,
        "input_size": 20,
        "num_layers": 2,
    },
    "medium": {
        "hidden_size": 128,
        "input_size": 20,
        "num_layers": 1,
    },
}


experiment_list = [
    (
        f"trainer_{trainer.__name__}_params_{param_name}", 
        trainer,
        str(run),
        param
    )
    for run in range(11, 21) 
    for trainer in [oneLSTM]
    for param_name, param in param_grid.items()
]

In [9]:
len(experiment_list)

40

In [ ]:
# def run_experiment_process(experiment):
for experiment in experiment_list:
    print(
        "Time:", datetime.datetime.now().isoformat(),
        "Experiment:", experiment[0]
    )
    try:
        run_experiment(*experiment)
    except Exception as e:
        print("Error occured, skipping...\n", repr(e))

Time: 2023-04-23T10:53:22.288482 Experiment: trainer_oneLSTM_params_small
Running experiment for  experiments_rnn/trainer_oneLSTM_params_small_run_11
Report exists already for experiments_rnn/trainer_oneLSTM_params_small_run_11. Skipping...
Time: 2023-04-23T10:53:22.288565 Experiment: trainer_oneLSTM_params_smalldouble
Running experiment for  experiments_rnn/trainer_oneLSTM_params_smalldouble_run_11
Report exists already for experiments_rnn/trainer_oneLSTM_params_smalldouble_run_11. Skipping...
Time: 2023-04-23T10:53:22.288601 Experiment: trainer_oneLSTM_params_tinydouble
Running experiment for  experiments_rnn/trainer_oneLSTM_params_tinydouble_run_11
Report exists already for experiments_rnn/trainer_oneLSTM_params_tinydouble_run_11. Skipping...
Time: 2023-04-23T10:53:22.288637 Experiment: trainer_oneLSTM_params_medium
Running experiment for  experiments_rnn/trainer_oneLSTM_params_medium_run_11
Report exists already for experiments_rnn/trainer_oneLSTM_params_medium_run_11. Skipping...


Epoch: 84, Accuracy on validation set: 0.7014681892332789
Epoch: 85, Accuracy on validation set: 0.7226753670473083
Epoch: 86, Accuracy on validation set: 0.7132499546855174
Epoch: 87, Accuracy on validation set: 0.7020119630233823
Epoch: 88, Accuracy on validation set: 0.7045495740438644
Epoch: 89, Accuracy on validation set: 0.7056371216240711
Epoch: 90, Accuracy on validation set: 0.7088997643646909
Epoch: 91, Accuracy on validation set: 0.7148812760558274
Epoch: 92, Accuracy on validation set: 0.7123436650353453
Epoch: 93, Accuracy on validation set: 0.7230378829073772
Epoch: 94, Accuracy on validation set: 0.7183251767264818
Epoch: 95, Accuracy on validation set: 0.7199564980967917
Epoch: 96, Accuracy on validation set: 0.723762914627515
Epoch: 97, Accuracy on validation set: 0.7090810222947254
Epoch: 98, Accuracy on validation set: 0.7127061808954142
Epoch: 99, Accuracy on validation set: 0.7257567518578938
Epoch: 100, Accuracy on validation set: 0.7188689505165851
Epoch: 101, Ac

Epoch: 88, Accuracy on validation set: 0.6837066473988439
Epoch: 89, Accuracy on validation set: 0.693099710982659
Epoch: 90, Accuracy on validation set: 0.6950867052023122
Epoch: 91, Accuracy on validation set: 0.6800939306358381
Epoch: 92, Accuracy on validation set: 0.6721459537572254
Epoch: 93, Accuracy on validation set: 0.6829841040462428
Epoch: 94, Accuracy on validation set: 0.6970736994219653
Epoch: 95, Accuracy on validation set: 0.7095375722543352
Epoch: 96, Accuracy on validation set: 0.6773843930635838
Epoch: 97, Accuracy on validation set: 0.6950867052023122
Epoch: 98, Accuracy on validation set: 0.7003251445086706
Epoch: 99, Accuracy on validation set: 0.7019508670520231
Epoch: 100, Accuracy on validation set: 0.6887644508670521
Epoch: 101, Accuracy on validation set: 0.6885838150289018
Epoch: 102, Accuracy on validation set: 0.7113439306358381
Epoch: 103, Accuracy on validation set: 0.6838872832369942
Epoch: 104, Accuracy on validation set: 0.6958092485549133
Epoch: 105

Epoch: 11, Accuracy on validation set: 0.5479501535127326
Epoch: 12, Accuracy on validation set: 0.5439768827885136
Epoch: 13, Accuracy on validation set: 0.5721509842875203
Epoch: 14, Accuracy on validation set: 0.5885858768286075
Epoch: 15, Accuracy on validation set: 0.5820841610980676
Epoch: 16, Accuracy on validation set: 0.5555354885316959
Epoch: 17, Accuracy on validation set: 0.5777496839443742
Epoch: 18, Accuracy on validation set: 0.5851544157486003
Epoch: 19, Accuracy on validation set: 0.6196496297634098
Epoch: 20, Accuracy on validation set: 0.6212750586960448
Epoch: 21, Accuracy on validation set: 0.6080910240202275
Epoch: 22, Accuracy on validation set: 0.6297634097886943
Epoch: 23, Accuracy on validation set: 0.6313888387213292
Epoch: 24, Accuracy on validation set: 0.637709951237132
Epoch: 25, Accuracy on validation set: 0.6093552465233881
Epoch: 26, Accuracy on validation set: 0.6411414123171393
Epoch: 27, Accuracy on validation set: 0.6648004334477153
Epoch: 28, Accu

Epoch: 17, Accuracy on validation set: 0.5283592771515477
Epoch: 18, Accuracy on validation set: 0.5392735730899982
Epoch: 19, Accuracy on validation set: 0.5392735730899982
Epoch: 20, Accuracy on validation set: 0.5534084809447128
Epoch: 21, Accuracy on validation set: 0.5797101449275363
Epoch: 22, Accuracy on validation set: 0.6001073537305421
Epoch: 23, Accuracy on validation set: 0.5639649311146896
Epoch: 24, Accuracy on validation set: 0.5562712470925031
Epoch: 25, Accuracy on validation set: 0.5602075505457148
Epoch: 26, Accuracy on validation set: 0.6056539631418858
Epoch: 27, Accuracy on validation set: 0.6047593487207014
Epoch: 28, Accuracy on validation set: 0.5961710502773304
Epoch: 29, Accuracy on validation set: 0.5977813562354625
Epoch: 30, Accuracy on validation set: 0.60368581141528
Epoch: 31, Accuracy on validation set: 0.6239040973340491
Epoch: 32, Accuracy on validation set: 0.6160314904276257
Epoch: 33, Accuracy on validation set: 0.6255144032921811
Epoch: 34, Accur

...

Epoch: 42, Accuracy on validation set: 0.6380279137212252
Epoch: 43, Accuracy on validation set: 0.6518035164038427
Epoch: 44, Accuracy on validation set: 0.6537973536342215
Epoch: 45, Accuracy on validation set: 0.637484139931122
Epoch: 46, Accuracy on validation set: 0.6503534529635672
Epoch: 47, Accuracy on validation set: 0.6376653978611564
Epoch: 48, Accuracy on validation set: 0.6537973536342215
Epoch: 49, Accuracy on validation set: 0.646547036432844
Epoch: 50, Accuracy on validation set: 0.650715968823636
Epoch: 51, Accuracy on validation set: 0.6760920790284575
Epoch: 52, Accuracy on validation set: 0.6612289287656336
Epoch: 53, Accuracy on validation set: 0.6612289287656336
Epoch: 54, Accuracy on validation set: 0.6855174913902483
Epoch: 55, Accuracy on validation set: 0.6537973536342215
Epoch: 56, Accuracy on validation set: 0.6565162225847381
Epoch: 57, Accuracy on validation set: 0.6547036432843937
Epoch: 58, Accuracy on validation set: 0.671741888707631
Epoch: 59, Accurac

Epoch: 183, Accuracy on validation set: 0.7342758745695124
Epoch: 184, Accuracy on validation set: 0.7408011600507522
Epoch: 185, Accuracy on validation set: 0.7500453144825087
Epoch: 186, Accuracy on validation set: 0.738626064890339
Epoch: 187, Accuracy on validation set: 0.736994743520029
Early stop
Time: 2023-04-23T14:48:02.312136 Experiment: trainer_oneLSTM_params_medium
Running experiment for  experiments_rnn/trainer_oneLSTM_params_medium_run_17
Epoch: 1, Accuracy on validation set: 0.2598154514203003
Epoch: 2, Accuracy on validation set: 0.2826126289126108
Epoch: 3, Accuracy on validation set: 0.3387009227428985
Epoch: 4, Accuracy on validation set: 0.3875520173692781
Epoch: 5, Accuracy on validation set: 0.42609010313008866
Epoch: 6, Accuracy on validation set: 0.4213859236475484
Epoch: 7, Accuracy on validation set: 0.4525058802243532
Epoch: 8, Accuracy on validation set: 0.4859779265424281
Epoch: 9, Accuracy on validation set: 0.49882395512936495
Epoch: 10, Accuracy on valida

Epoch: 4, Accuracy on validation set: 0.35769299659681175
Epoch: 5, Accuracy on validation set: 0.3618126455310765
Epoch: 6, Accuracy on validation set: 0.3709475192548809
Epoch: 7, Accuracy on validation set: 0.476983700519434
Epoch: 8, Accuracy on validation set: 0.45602722550600033
Epoch: 9, Accuracy on validation set: 0.4751925488088841
Epoch: 10, Accuracy on validation set: 0.44331004836109617
Epoch: 11, Accuracy on validation set: 0.5138814257567615
Epoch: 12, Accuracy on validation set: 0.5335840945728103
Epoch: 13, Accuracy on validation set: 0.5364499373096902
Epoch: 14, Accuracy on validation set: 0.5307182518359305
Epoch: 15, Accuracy on validation set: 0.5584811033494537
Epoch: 16, Accuracy on validation set: 0.5676159770732581
Epoch: 17, Accuracy on validation set: 0.5602722550600036
Epoch: 18, Accuracy on validation set: 0.569407128783808
Epoch: 19, Accuracy on validation set: 0.5484506537703744
Epoch: 20, Accuracy on validation set: 0.6032598961132007
Epoch: 21, Accuracy

Epoch: 145, Accuracy on validation set: 0.7288196310227476
Epoch: 146, Accuracy on validation set: 0.7372380440623321
Epoch: 147, Accuracy on validation set: 0.7297152068780226
Epoch: 148, Accuracy on validation set: 0.7338348558122872
Epoch: 149, Accuracy on validation set: 0.7245208669174279
Epoch: 150, Accuracy on validation set: 0.7452982267598065
Epoch: 151, Accuracy on validation set: 0.7356260075228371
Epoch: 152, Accuracy on validation set: 0.7316854737596274
Epoch: 153, Accuracy on validation set: 0.7426114991939817
Epoch: 154, Accuracy on validation set: 0.7293569765359126
Epoch: 155, Accuracy on validation set: 0.7336557406412323
Epoch: 156, Accuracy on validation set: 0.737954504746552
Epoch: 157, Accuracy on validation set: 0.7411785778255419
Epoch: 158, Accuracy on validation set: 0.7370589288912771
Epoch: 159, Accuracy on validation set: 0.723804406233208
Epoch: 160, Accuracy on validation set: 0.7320437041017375
Epoch: 161, Accuracy on validation set: 0.7289987461938026

Epoch: 15, Accuracy on validation set: 0.5326323422770123
Epoch: 16, Accuracy on validation set: 0.5197606961566352
Epoch: 17, Accuracy on validation set: 0.5279187817258884
Epoch: 18, Accuracy on validation set: 0.5634517766497462
Epoch: 19, Accuracy on validation set: 0.5886511965192168
Epoch: 20, Accuracy on validation set: 0.5650833937635968
Epoch: 21, Accuracy on validation set: 0.5335387962291516
Epoch: 22, Accuracy on validation set: 0.536620739666425
Epoch: 23, Accuracy on validation set: 0.588469905728789
Epoch: 24, Accuracy on validation set: 0.5935460478607687
Epoch: 25, Accuracy on validation set: 0.5988034807831762
Epoch: 26, Accuracy on validation set: 0.5911892675852066
Epoch: 27, Accuracy on validation set: 0.5917331399564902
Epoch: 28, Accuracy on validation set: 0.6009789702683104
Epoch: 29, Accuracy on validation set: 0.5937273386511965
Epoch: 30, Accuracy on validation set: 0.5610949963741841
Epoch: 31, Accuracy on validation set: 0.6006163886874547
Epoch: 32, Accur

In [ ]:
# from multiprocessing import Pool

# def f(x):
#     return x*x

# if __name__ == '__main__':
#     with Pool(3) as p:
#         print(p.map(run_experiment_process, experiment_list))